In [1]:
!which python

/Users/yevgeniykim/code/SchemesSG_v3/dataset_worfklow/venv/bin/python


In [2]:
# Setup
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd


# Use a service account to connect to firestore.
cred = credentials.Certificate("creds.json")

app = firebase_admin.initialize_app(cred)

db = firestore.client()


In [4]:
# Get all documents from the collection
docs = db.collection("schemes").stream()
schemes_df = pd.DataFrame([{**scheme.to_dict(), "scheme_id": scheme.id} for scheme in docs])


In [5]:
schemes_df

,Scheme Type,Scheme,Who's it for,Agency,Description,scraped_text,What it gives,Link,Image,search_booster(WL),scheme_id
0,"Low Income,Healthcare",Subsidised Haemodialysis and Portable Subsidy ...,"Kidney patients,Cancer patients,Stroke patient...",Kidney Dialysis Foundation (KDF),The subsidised haemodialysis programme at KDF ...,Subsidised HaemodialysisToggle navigationHomeA...,FInancial assistance for dialysis,https://www.kdf.org.sg/subsidised-haemodialysis,https://chidnast.sirv.com/SchemesSG/KDF.jpg,dialysis,023KgLCp3ZVORpeLMDLX
1,Housing,Public Rental Scheme,"Low income,Need shelter",Housing and Development Board (HDB),Flats under the Public Rental Scheme are heavi...,HDB | Bad RequestHomeBad RequestWe couldn't di...,"Shelter,Heavily subsidised housing",https://www.hdb.gov.sg/cs/infoweb/residential/...,https://chidnast.sirv.com/SchemesSG/hdb.jpg,"No money to pay for housing, shelter, homeless...",0GxV07IafFzHnSLLKlXn
2,"Foreign Domestic Workers/maids,Women",Migrant Domestic Workers Care Group,Foreign Domestic Workers/maids,Blessed Grace Social Services,Our volunteers spend time with migrant domesti...,MIGRANT DOMESTIC WORKERS (MDWs) | BGSStop of p...,"Counselling,Emotional care,Casework,Befriending",https://www.bgss.org.sg/migrant-domestic-worke...,https://timinone.sirv.com/Shared/SchemesSG-fro...,"no friends in singapore, new to singapore, mee...",0HTkEEoWEH8hUAexf5aC
3,"Youth,Children,Special Needs",SHINE Children & Youth Services,"Children,Youth,Youth with special needs",SHINE Children & Youth Services,"SHINE servces vulnerable children, youth and t...",Home | SHINE Children and Youth Servicestop of...,"Youth services,Educational programmes,Mental h...",https://www.shine.org.sg/,https://chidnast.sirv.com/SchemesSG/shine.jpg,"my child has learning needs, my child has lear...",0PwdzpDneXGZ435VnCkv
4,Housing,Fresh Start Housing Scheme,Low income families,Housing and Development Board (HDB),The Fresh Start Housing Scheme (Fresh Start) a...,HDB | Bad RequestHomeBad RequestWe couldn't di...,"Financial assistance,Housing assistance",https://www.hdb.gov.sg/residential/buying-a-fl...,https://chidnast.sirv.com/SchemesSG/hdb.jpg,"Need money to tide over difficulties, money fo...",0ayHvAOXQi92JsFWTlb9
...,...,...,...,...,...,...,...,...,...,...,...
295,"PWD,Family,Transport,Healthcare,Mental Health",Rehabilitation Centre and Respite Centre,"PWD,Elderly,Caregivers",Abilities Beyond Limitations and Expectations ...,"Provides Occupational Therapy, Physiotherapy a...",Abilities Beyond Limitations and Expectations ...,"Occupational therapy,Physiotherapy,Educational...",https://www.able-sg.org/,https://chidnast.sirv.com/SchemesSG/able.jpg,"rehabilitation, occupational therapy, physioth...",zBbxmhxtkXOAHgVBkFva
296,"Elderly,Low income,Healthcare","Community Care, Home Care & TCM for Elderly","Elderly, Low income",Kwong Wai Shiu Hospital,KWSH is a charitable healthcare organisation c...,Services Overview - Kwong Wai Shiu HospitalPle...,"Traditional Chinese Medicine, Rehabilitation, ...",https://www.kwsh.org.sg/ourservices/,https://chidnast.sirv.com/SchemesSG/kwsh.jpg,"need a place for my old parents to stay, nursi...",zCEgTYhLzjop9KpPBmcI
297,"Family,Low Income",ComCare Short-To-Medium-Term Assistance (SMTA),Low income,Ministry of Social and Family Development (MSF),ComCare Short-to-Medium-Term Assistance (SMTA)...,SupportGoWhereYou need to enable JavaScript to...,Financial assistance,https://supportgowhere.life.gov.sg/schemes/COM...,https://chidnast.sirv.com/SchemesSG/msf.jpg,"Need money for daily needs, low income, poor",zQ6czovdyGxYBi98i5tf
298,"Healthcare, Elderly",Free Dental Services (Clinic),"Dental services,Healthcare,Medical,Low Income,...",Buddhist Compassion Relief Tzu-Chi Foundation ...,"Provides free General Medicine, Traditional Ch...",Free Clinic (Dental Services) - Tzu-Chi Founda...,Financial assistance for HIV patients,https://www.tzuchi.org.sg/en/our-missions/medi...,https://chidnast.sirv.com/SchemesSG/tzuchi.jpg,HIV and AI

### Import schemes data and preprocess clean ###

1. Load schemes data using enriched schemes-updated-with-text.csv
2. Define preprocessing function to clean and lemmatize a text
3. Create desc_booster column combining Description, search_booster(WL), Who's it for, What it gives, Scheme Type
4. Apply preprocessing to desc_booster

In [7]:
#import pandas as pd
# df = pd.read_csv("schemes 11 nov.csv")
df = schemes_df
df.head()

,Scheme Type,Scheme,Who's it for,Agency,Description,scraped_text,What it gives,Link,Image,search_booster(WL),scheme_id
0,"Low Income,Healthcare",Subsidised Haemodialysis and Portable Subsidy ...,"Kidney patients,Cancer patients,Stroke patient...",Kidney Dialysis Foundation (KDF),The subsidised haemodialysis programme at KDF ...,Subsidised HaemodialysisToggle navigationHomeA...,FInancial assistance for dialysis,https://www.kdf.org.sg/subsidised-haemodialysis,https://chidnast.sirv.com/SchemesSG/KDF.jpg,dialysis,023KgLCp3ZVORpeLMDLX
1,Housing,Public Rental Scheme,"Low income,Need shelter",Housing and Development Board (HDB),Flats under the Public Rental Scheme are heavi...,HDB | Bad RequestHomeBad RequestWe couldn't di...,"Shelter,Heavily subsidised housing",https://www.hdb.gov.sg/cs/infoweb/residential/...,https://chidnast.sirv.com/SchemesSG/hdb.jpg,"No money to pay for housing, shelter, homeless...",0GxV07IafFzHnSLLKlXn
2,"Foreign Domestic Workers/maids,Women",Migrant Domestic Workers Care Group,Foreign Domestic Workers/maids,Blessed Grace Social Services,Our volunteers spend time with migrant domesti...,MIGRANT DOMESTIC WORKERS (MDWs) | BGSStop of p...,"Counselling,Emotional care,Casework,Befriending",https://www.bgss.org.sg/migrant-domestic-worke...,https://timinone.sirv.com/Shared/SchemesSG-fro...,"no friends in singapore, new to singapore, mee...",0HTkEEoWEH8hUAexf5aC
3,"Youth,Children,Special Needs",SHINE Children & Youth Services,"Children,Youth,Youth with special needs",SHINE Children & Youth Services,"SHINE servces vulnerable children, youth and t...",Home | SHINE Children and Youth Servicestop of...,"Youth services,Educational programmes,Mental h...",https://www.shine.org.sg/,https://chidnast.sirv.com/SchemesSG/shine.jpg,"my child has learning needs, my child has lear...",0PwdzpDneXGZ435VnCkv
4,Housing,Fresh Start Housing Scheme,Low income families,Housing and Development Board (HDB),The Fresh Start Housing Scheme (Fresh Start) a...,HDB | Bad RequestHomeBad RequestWe couldn't di...,"Financial assistance,Housing assistance",https://www.hdb.gov.sg/residential/buying-a-fl...,https://chidnast.sirv.com/SchemesSG/hdb.jpg,"Need money to tide over difficulties, money fo...",0ayHvAOXQi92JsFWTlb9


In [6]:
import spacy
import re

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

def preprocessing(sentence):
    # Text cleaning steps from spacy_tokenizer
    sentence = re.sub('\'', '', sentence)  # Remove distracting single quotes
    sentence = re.sub(' +', ' ', sentence)  # Replace extra spaces
    sentence = re.sub(r'\n: \'\'.*', '', sentence)  # Remove specific unwanted lines
    sentence = re.sub(r'\n!.*', '', sentence)
    sentence = re.sub(r'^:\'\'.*', '', sentence)
    sentence = re.sub(r'\n', ' ', sentence)  # Replace non-breaking new lines with space

    # Tokenization and further processing with spaCy
    doc = nlp(sentence)
    tokens = []
    for token in doc:
        # Check if the token is a stopword or punctuation
        if token.is_stop or token.is_punct:
            continue
        # Check for numeric tokens or tokens longer than 2 characters
        if token.like_num or len(token.text) > 2:
            # Lemmatize (handling pronouns) and apply lowercase
            lemma = token.lemma_.lower().strip() if token.lemma_ != "-PRON-" else token.lower_
            tokens.append(lemma)

    # Further clean up to remove any introduced extra spaces
    processed_text = ' '.join(tokens)
    processed_text = re.sub(' +', ' ', processed_text)

    return processed_text


In [8]:
# Build desc_booster by concatenating non-null values
def build_desc_booster(row):
    components = []

    # Check each column and add non-null values
    if pd.notna(row['Scheme']):
        components.append(str(row['Scheme']))
    if pd.notna(row['Agency']):
        components.append(str(row['Agency']))
    if pd.notna(row['Description']):
        components.append(str(row['Description']))
    if pd.notna(row['search_booster(WL)']):
        components.append(str(row['search_booster(WL)']))
    if pd.notna(row["Who's it for"]):
        components.append(str(row["Who's it for"]))
    if pd.notna(row['What it gives']):
        components.append(str(row['What it gives']))
    if pd.notna(row['Scheme Type']):
        components.append(str(row['Scheme Type']))

    # Join all non-null components with spaces
    return ' '.join(components)

# Apply build_desc_booster to each row and store in desc_booster column
df['desc_booster'] = df.apply(build_desc_booster, axis=1)

# Print a sample row to verify
print("Sample desc_booster value:")
print(df['desc_booster'].iloc[0])

Sample desc_booster value:
Subsidised Haemodialysis and Portable Subsidy Programme Kidney Dialysis Foundation (KDF) The subsidised haemodialysis programme at KDF serves needy kidney patients in Singapore, regardless of race or religion. The Portable Subsidy Programme serves to help needy patients who require greater medical attention that KDF is unable to provide in our step-down community care environment. Under the Portable Subsidy Programme, high-dependency dialysis patients can apply for financial assistance from KDF for their dialysis treatment at a private dialysis centre. dialysis Kidney patients,Cancer patients,Stroke patients,Diabetes patients FInancial assistance for dialysis Low Income,Healthcare


In [9]:
# Get rows where desc_booster is NA
na_rows = df[df['desc_booster'].isna()]
print("Full rows where desc_booster is NA:")
na_rows

Full rows where desc_booster is NA:


,Scheme Type,Scheme,Who's it for,Agency,Description,scraped_text,What it gives,Link,Image,search_booster(WL),scheme_id,desc_booster


df['clean_text'] = df['desc_booster'].apply(lambda x: preprocessing(x))
df.head()

### Generate enhanced Document embeddings ###

1. Load pretrained all-mpnet-base-v2 model and tokenizer
2. Compute embedding for clean_text column , perform pooling and normalize embeddings
3. Create FAISS index for embeddings
4. Save model, tokenizer, embeddings, faiss index as files

In [10]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')


/Users/yevgeniykim/code/SchemesSG_v3/dataset_worfklow/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/yevgeniykim/code/SchemesSG_v3/dataset_worfklow/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
# Tokenize sentences
encoded_input = tokenizer(df['desc_booster'].tolist(), padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)

Sentence embeddings:
tensor([[ 0.0449,  0.0022, -0.0193,  ..., -0.0159, -0.0402,  0.0120],
        [-0.0130, -0.0463,  0.0133,  ..., -0.0136, -0.0619,  0.0378],
        [-0.0435,  0.0252, -0.0103,  ...,  0.0062,  0.0433,  0.0100],
        ...,
        [-0.0443, -0.0945, -0.0231,  ...,  0.0063, -0.0234, -0.0023],
        [ 0.0287,  0.0096,  0.0044,  ..., -0.0109,  0.0049, -0.0167],
        [-0.0385, -0.0557,  0.0070,  ..., -0.0179, -0.0192,  0.0075]])


In [15]:
import faiss
import numpy as np

embeddings = sentence_embeddings
# Convert embeddings to np.float32 as required by FAISS
embeddings = np.array(embeddings).astype('float32')

# Create a FAISS index
dimension = embeddings.shape[1]  # Dimension of the embeddings
index = faiss.IndexFlatL2(dimension)  # Using the L2 distance for similarity
index.add(embeddings)  # Adding the embeddings to the index


In [16]:
# Create a mapping between FAISS index and `scheme_id`
index_to_scheme_id = dict(enumerate(df['scheme_id']))

# Save this mapping to a file or database for later retrieval
# For example, save to a JSON file
import json
with open('index_to_scheme_id.json', 'w') as f:
    json.dump(index_to_scheme_id, f)


In [17]:
import numpy as np

# Assuming `model` is your PyTorch model and `tokenizer` is the Hugging Face tokenizer
model_save_path = './models/schemesv2-torch-allmpp-model'
tokenizer_save_path = './models/schemesv2-torch-allmpp-tokenizer'
embeddings_save_name = './models/schemesv2-your_embeddings.npy'
index_save_name = './models/schemesv2-your_index.faiss'


# Save the embeddings and index to disk
np.save(embeddings_save_name, embeddings)
faiss.write_index(index, index_save_name)

# Save model
model.save_pretrained(model_save_path)

# Save tokenizer
tokenizer.save_pretrained(tokenizer_save_path)


('./models/schemesv2-torch-allmpp-tokenizer/tokenizer_config.json',
 './models/schemesv2-torch-allmpp-tokenizer/special_tokens_map.json',
 './models/schemesv2-torch-allmpp-tokenizer/vocab.txt',
 './models/schemesv2-torch-allmpp-tokenizer/added_tokens.json',
 './models/schemesv2-torch-allmpp-tokenizer/tokenizer.json')

### Sample run

1. Load models, tokenizer, embeddings, faiss index
2. Define split_query_into_needs function to split a query into distinct needs
3. 

In [19]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import numpy as np
import faiss


model_save_path = './models/schemesv2-torch-allmpp-model'
tokenizer_save_path = './models/schemesv2-torch-allmpp-tokenizer'
embeddings_save_name = './models/schemesv2-your_embeddings.npy'
index_save_name = './models/schemesv2-your_index.faiss'

# Load the FAISS index-to-scheme_id mapping
with open('index_to_scheme_id.json', 'r') as f:
    index_to_scheme_id = json.load(f)

# Load model and tokenizer at startup
model = AutoModel.from_pretrained(model_save_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_path)

# Load the embeddings and index
embeddings = np.load(embeddings_save_name)
index = faiss.read_index(index_save_name)

In [24]:
import faiss
import numpy as np

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Now, you can use `index` for similarity searches with new user queries
def search_similar_items(query_text, full_query, top_k=10):

    # preproc = preprocessing(query_text)
    preproc = query_text
    # Compute embedding for the query text
    # query_embedding = model.encode([preproc])

    # Tokenize text
    encoded_input = tokenizer([preproc], padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    query_embedding = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    query_embedding = F.normalize(query_embedding, p=2, dim=1)

    query_embedding = np.array(query_embedding).astype('float32')

    # Perform the search
    distances, indices = index.search(query_embedding, top_k)
    similarity_scores =  np.exp(-distances)

    # Retrieve the scheme_ids using the FAISS indices
    retrieved_scheme_ids = [index_to_scheme_id[str(idx)] for idx in indices[0]]

    # Fetch scheme details from Firestore
    scheme_details = []
    for scheme_id in retrieved_scheme_ids:
        doc_ref = db.collection("schemes").document(scheme_id)
        doc = doc_ref.get()
        if doc.exists:
            scheme_details.append(doc.to_dict())

    # Convert scheme details to a DataFrame
    scheme_df = pd.DataFrame(scheme_details)

    # Add similarity scores to the DataFrame
    results = pd.concat(
        [scheme_df.reset_index(drop=True), pd.DataFrame(similarity_scores[0], columns=['Similarity']).reset_index(drop=True)],
        axis=1
    )

    # Add the query to the results and sort by similarity
    results['query'] = full_query
    results = results.sort_values(['Similarity'], ascending=False)


    # similar_items = pd.DataFrame([df.iloc[indices[0]], distances[0], similarity_scores[0]])
    # Retrieve the most similar items
    #similar_items = df.iloc[indices[0]][['Scheme', 'Agency', 'Description']]

    #results = pd.concat([similar_items.reset_index(drop=True), pd.DataFrame(similarity_scores[0]).reset_index(drop=True)], axis=1)
    #results = results.set_axis(['Scheme', 'Agency', 'Description', 'Similarity'], axis=1)
    #results['query'] = full_query
    #results = results.sort_values(['Similarity'], ascending=False)

    return results


In [25]:

def combine_and_aggregate_results(needs, user_query):
    # DataFrame to store combined results
    combined_results = pd.DataFrame(columns=['Scheme', 'Agency', 'Description', 'Similarity', 'query'])

    # Process each need
    for need in needs:
        # Get the results for the current need
        current_results = search_similar_items(need, user_query)
        # Combine with the overall results
        combined_results = pd.concat([combined_results, current_results], ignore_index=True)

    # Handle duplicates: Aggregate similarity for duplicates and drop duplicates
    aggregated_results = combined_results.groupby(['Scheme', 'Agency', 'Description', 'query'], as_index=False).agg({
        'Similarity': 'mean'  # Adjust this function as needed to aggregate similarity scores appropriately
    })

    # Sort by similarity in descending order
    sorted_results = aggregated_results.sort_values(by='Similarity', ascending=False).reset_index(drop=True)

    return sorted_results


In [26]:

def extract_needs_based_on_conjunctions(sentence):
    """Extract distinct needs based on coordinating conjunctions."""
    doc = nlp(sentence)
    needs = []
    current_need_tokens = []

    for token in doc:
        # If the token is a coordinating conjunction (e.g., 'and') and not at the start of the sentence,
        # consider the preceding tokens as one distinct need.
        if token.text.lower() in ['and', 'or'] and token.i != 0:
            if current_need_tokens:  # Ensure there's content before the conjunction
                needs.append(" ".join([t.text for t in current_need_tokens]))
                current_need_tokens = []  # Reset for the next need
        else:
            current_need_tokens.append(token)

    # Add the last accumulated tokens as a need, if any.
    if current_need_tokens:
        needs.append(" ".join([t.text for t in current_need_tokens]))

    return needs

def split_query_into_needs(query):
    """Split the query into sentences and then extract needs focusing on conjunctions."""
    sentences = split_into_sentences(query)
    all_needs = []
    for sentence in sentences:
        needs_in_sentence = extract_needs_based_on_conjunctions(sentence)
        all_needs.extend(needs_in_sentence)
    return all_needs

# Helper function to split the query into sentences
def split_into_sentences(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

# Example usage
query = "my client needs a blood pressure monitor"
distinct_needs = split_query_into_needs(query)
print(f"Distinct needs: {distinct_needs}")

print(f"Distinct needs preproc : {[preprocessing(x) for x in distinct_needs]}")


Distinct needs: ['my client needs a blood pressure monitor']
Distinct needs preproc : ['client need blood pressure monitor']


In [27]:
# user_query = "My client needs assistance in getting diapers and milk powder as she is from a low-income family which has only one sole breadwinner"
user_query = "I am pregnant teen suffering from depression and family abuse"
# user_query = "My client needs assistance as a dialysis patient. She is also in need of a job and financial support after COVID 19 has caused her to be retrenched"

split_query = split_query_into_needs(user_query)
# split_query = split_query_into_needs("I am a 31 year old married with one kid in need of more money")
print(split_query)

# proc_split_query = [preprocessing(x) for x in split_query]
# print(proc_split_query)
# Example usage
# final_results = combine_and_aggregate_results(proc_split_query, user_query)
final_results = combine_and_aggregate_results(split_query, user_query)

final_results

['I am pregnant teen suffering from depression', 'family abuse']


/var/folders/xv/8hpschc94mq0cfdl4h76xmtc0000gn/T/ipykernel_32423/1410995992.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_results = pd.concat([combined_results, current_results], ignore_index=True)


,Scheme,Agency,Description,query,Similarity
0,Sexual Assault Care Centre,Association of Women for Action and Research (...,"The Care Centre provides safe, free and confid...",I am pregnant teen suffering from depression a...,0.392225
1,Pregnancy Crisis and Support,Pregnancy Crisis and Support,"For emotional support, guidance, help and refe...",I am pregnant teen suffering from depression a...,0.376997
2,Babes - A Helping Hand for Pregnant Teens,Babes - A Helping Hand for Pregnant Teens,Staff will discuss the various options availab...,I am pregnant teen suffering from depression a...,0.375273
3,Women and Children's Shelters,Casa Raudha Ltd (CRL),Provides safe accommodation for victims of dom...,I am pregnant teen suffering from depression a...,0.357147
4,HCSA Dayspring Residential Treatment Centre,HCSA Community Services,Therapeutic Group Home service model for teena...,I am pregnant teen suffering from depression a...,0.356041
5,Anglican Family Centre (AFC),Singapore Anglican Community Services (SACS),Serves women and children who are victims of d...,I am pregnant teen suffering from depression a...,0.355847
6,National Anti-Violence & Sexual Harrassment He...,Ministry of Social and Family Development (MSF),"24/7 hotline for persons to seek information, ...",I am pregnant teen suffering from depression a...,0.354004
7,Various services,Big Love Child Protection Specialist Centre,Casework management; child protection; home-ba...,I am pregnant teen suffering from depression a...,0.350162
8,PAVE Integrated Services for Individual and Fa...,PAVE,PAVE is an integrated service for family viole...,I am pregnant teen suffering from depression a...,0.343552
9,SAFE Programme (Stop Abuse in Families),Trans Family Services,TRANS SAFE Centre is a Family Violence Special...,I am pregnant teen suffering from depression a...,0.342666


### Testing the existing queries data

In [ ]:
dfq = pd.read_csv("../raw_data/schemes-queries.csv")
dfq.describe()

In [ ]:

import pandas as pd

dfq_unique = dfq.drop_duplicates(subset=["Query"])
dfq_unique = dfq_unique.dropna(subset=["Query"])

# Assume you have a list of queries to test
queries = dfq_unique['Query']

# Initialize an empty DataFrame for concatenating all results
all_results_df = pd.DataFrame()

for query in queries:
    # Perform the search
    split_query = split_query_into_needs(query)
    #proc_split_query = [preprocessing(x) for x in split_query]
    split_query = split_query
    results = combine_and_aggregate_results(proc_split_query, query)
    top_10 = results.head(10)

    # Concatenate the current results with the overall results DataFrame
    all_results_df = pd.concat([all_results_df, top_10], ignore_index=True)


# Save the overall results DataFrame to a CSV file
all_results_df.to_csv('./overall_search_results_transformers_laiss.csv', index=False)

print("All search results saved to 'overall_search_results_transformers_laiss.csv'")